In [24]:
import os
import torch
import numpy as np
import pandas as pd
import sys
os.chdir('~/projects/LLaVA')
current_path = os.getcwd()
sys.path.append(current_path + '/data-engine')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from utils import set_env
set_env(seed=1234)

from torch.utils.data import Dataset, DataLoader
from utils import ChartFeatureDataset
from models import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from utils import FocalLoss

In [25]:
def train(data, pretrained_features, feature_column_name, criterion, epochs=20, batch_size=2048):

    num_classes = len(data[feature_column_name].unique())
    train_data, test_data, train_features, test_features = train_test_split(
        data, pretrained_features, test_size=0.2, random_state=1234)
    train_dataset = ChartFeatureDataset(train_data, train_features, feature_column_name)
    test_dataset = ChartFeatureDataset(test_data, test_features, feature_column_name)
    train_dataloader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)
    classifier = MLPClassifier(
        input_dim=pretrained_features.shape[1], num_classes=num_classes).to(device)  

    
    optimizer = torch.optim.Adam(classifier.parameters(), lr=2e-3)
    
    
    
    for epoch in tqdm(range(epochs)):
        classifier.train()
        total_loss = 0
        train_preds, train_labels_list = [], []
        for features, labels in train_dataloader:
            labels = labels.to(device)
            features = features.to(device)
            optimizer.zero_grad()
            
            outputs = classifier(features)
            _, predicted = torch.max(outputs, 1)
            # loss = criterion(outputs, F.one_hot(labels, num_classes=num_classes).float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            train_preds.extend(predicted.cpu().numpy())
            train_labels_list.extend(labels.cpu().numpy())
            
        train_accuracy = accuracy_score(train_labels_list, train_preds)
  
        #eval
        classifier.eval()
        test_preds, test_labels_list = [], []
        with torch.no_grad():
            for features, labels in test_dataloader:
                labels = labels.to(device)
                features = features.to(device)

                outputs = classifier(features)
                _, predicted = torch.max(outputs, 1)
                test_preds.extend(predicted.cpu().numpy())
                test_labels_list.extend(labels.cpu().numpy())
        test_accuracy = accuracy_score(test_labels_list, test_preds)
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}, Train Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}")
        
        # Report
    print("Classification Report on Test Data:")
    print(classification_report(test_labels_list, test_preds, digits=4))
    return classifier

In [26]:
data = pd.read_csv('data-engine/precomputed_features/classification_annotation_30k_5_attributes.csv')
clip_features = np.load('data-engine/precomputed_features/annotated-dataset-clip-feature.npy')
resnet_features = np.load('data-engine/precomputed_features/annotated-dataset-resnet-feature.npy')
convnext_features = np.load('data-engine/precomputed_features/annotated-dataset-convnext-feature.npy')
feature_column_name = 'number-annotation' # label, Trend, Layout, number annotation, grouping

## Focal loss

In [9]:
feature_column_name = 'number-annotation'
criterion = FocalLoss()
features = convnext_features
classifier = train(data, features, feature_column_name, criterion)

  5%|▌         | 1/20 [00:04<01:23,  4.40s/it]

Epoch 1, Loss: 0.4237421475923978, Train Accuracy: 0.7919798556825015, Test Accuracy: 0.8242633794347565


 10%|█         | 2/20 [00:07<01:03,  3.55s/it]

Epoch 2, Loss: 0.2935067048439613, Train Accuracy: 0.8647399278412508, Test Accuracy: 0.8735718580877931


 15%|█▌        | 3/20 [00:10<00:54,  3.18s/it]

...

Epoch 18, Loss: 0.10994671285152435, Train Accuracy: 0.9620039085989176, Test Accuracy: 0.9499398677089597


 95%|█████████▌| 19/20 [01:00<00:03,  3.15s/it]

Epoch 19, Loss: 0.10841299536136481, Train Accuracy: 0.962079073962718, Test Accuracy: 0.9559530968129886


100%|██████████| 20/20 [01:03<00:00,  3.18s/it]

Epoch 20, Loss: 0.10502622047295937, Train Accuracy: 0.9640333734215274, Test Accuracy: 0.9553517739025857
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9748    0.9707    0.9728      5470
           1     0.8672    0.8841    0.8756      1182

    accuracy                         0.9554      6652
   macro avg     0.9210    0.9274    0.9242      6652
weighted avg     0.9557    0.9554    0.9555      6652



In [27]:
feature_column_name = 'number-annotation'
criterion = FocalLoss()
features = clip_features
classifier = train(data, features, feature_column_name, criterion)

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:02<00:51,  2.71s/it]

Epoch 1, Loss: 0.41647157302269566, Train Accuracy: 0.8117483463619964, Test Accuracy: 0.8377931449188214


 10%|█         | 2/20 [00:05<00:51,  2.89s/it]

Epoch 2, Loss: 0.30598413714995754, Train Accuracy: 0.8613574864702345, Test Accuracy: 0.8858989777510523


...

Epoch 18, Loss: 0.12772534386469767, Train Accuracy: 0.9572684906794949, Test Accuracy: 0.940619362597715


 95%|█████████▌| 19/20 [00:49<00:02,  2.40s/it]

Epoch 19, Loss: 0.12552603162251985, Train Accuracy: 0.9580953096812989, Test Accuracy: 0.9407696933253157


100%|██████████| 20/20 [00:51<00:00,  2.59s/it]

Epoch 20, Loss: 0.12237609464388627, Train Accuracy: 0.9585838845460012, Test Accuracy: 0.9409200240529164
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9591    0.9695    0.9643      5470
           1     0.8513    0.8088    0.8295      1182

    accuracy                         0.9409      6652
   macro avg     0.9052    0.8891    0.8969      6652
weighted avg     0.9400    0.9409    0.9403      6652



In [28]:
feature_column_name = 'number-annotation'
criterion = FocalLoss()
features = resnet_features
classifier = train(data, features, feature_column_name, criterion)

  5%|▌         | 1/20 [00:03<00:57,  3.00s/it]

Epoch 1, Loss: 0.6761453449726105, Train Accuracy: 0.8149052916416115, Test Accuracy: 0.8400481058328322


 10%|█         | 2/20 [00:05<00:51,  2.89s/it]

Epoch 2, Loss: 0.3781271829054906, Train Accuracy: 0.8507215874924835, Test Accuracy: 0.8538785327720986


 15%|█▌        | 3/20 [00:08<00:46,  2.76s/it]

...

Epoch 18, Loss: 0.2019736778277617, Train Accuracy: 0.920625375826819, Test Accuracy: 0.9189717378232111


 95%|█████████▌| 19/20 [00:51<00:02,  2.86s/it]

Epoch 19, Loss: 0.19795447473342603, Train Accuracy: 0.9232561635598316, Test Accuracy: 0.921677690920024


100%|██████████| 20/20 [00:54<00:00,  2.73s/it]

Epoch 20, Loss: 0.19450021707094634, Train Accuracy: 0.925210463018641, Test Accuracy: 0.9174684305472038
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9269    0.9766    0.9511      5470
           1     0.8560    0.6438    0.7349      1182

    accuracy                         0.9175      6652
   macro avg     0.8915    0.8102    0.8430      6652
weighted avg     0.9143    0.9175    0.9127      6652



## MSE loss

In [30]:
def train(data, pretrained_features, feature_column_name, criterion):
    # data = pd.read_csv('filtering/annotation_18_30k.csv')
    # You are expected to extract corresponding features with feature_extraction.ipynb
    
    num_classes = len(data[feature_column_name].unique())
    # print(data[feature_column_name].unique())

    train_data, test_data, train_features, test_features = train_test_split(
        data, pretrained_features, test_size=0.2, random_state=1234)
    train_dataset = ChartFeatureDataset(train_data, train_features, feature_column_name)
    test_dataset = ChartFeatureDataset(test_data, test_features, feature_column_name)
    train_dataloader = DataLoader(
        train_dataset, batch_size=2048, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(
        test_dataset, batch_size=2048, shuffle=False, num_workers=8)
    classifier = MLPClassifier(
        input_dim=pretrained_features.shape[1], num_classes=num_classes).to(device)  

    
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
    
    epochs = 20
    
    for epoch in tqdm(range(epochs)):
        classifier.train()
        total_loss = 0
        train_preds, train_labels_list = [], []
        for features, labels in train_dataloader:
            labels = labels.to(device)
            features = features.to(device)
            optimizer.zero_grad()
            
            outputs = classifier(features)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, F.one_hot(labels, num_classes=num_classes).float())
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            train_preds.extend(predicted.cpu().numpy())
            train_labels_list.extend(labels.cpu().numpy())
            
        train_accuracy = accuracy_score(train_labels_list, train_preds)
  
        #eval
        classifier.eval()
        test_preds, test_labels_list = [], []
        with torch.no_grad():
            for features, labels in test_dataloader:
                labels = labels.to(device)
                features = features.to(device)

                outputs = classifier(features)
                _, predicted = torch.max(outputs, 1)
                test_preds.extend(predicted.cpu().numpy())
                test_labels_list.extend(labels.cpu().numpy())
        test_accuracy = accuracy_score(test_labels_list, test_preds)
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}, Train Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}")
        
        # Report
    print("Classification Report on Test Data:")
    print(classification_report(test_labels_list, test_preds, digits=4))
    return classifier

In [18]:
criterion = nn.MSELoss()
feature_column_name = 'number-annotation' # label, Trend, Layout, number annotation, grouping
features = convnext_features
classifier = train(data, features, feature_column_name, criterion)

  5%|▌         | 1/20 [00:03<01:00,  3.18s/it]

Epoch 1, Loss: 0.17285589529917791, Train Accuracy: 0.8227600721587492, Test Accuracy: 0.8907095610342755


 10%|█         | 2/20 [00:05<00:52,  2.90s/it]

Epoch 2, Loss: 0.08631693285245162, Train Accuracy: 0.9076217678893566, Test Accuracy: 0.9076969332531569


 15%|█▌        | 3/20 [00:08<00:48,  2.85s/it]

...

Epoch 18, Loss: 0.03889151605275961, Train Accuracy: 0.9572684906794949, Test Accuracy: 0.9490378833433554


 95%|█████████▌| 19/20 [00:53<00:02,  2.57s/it]

Epoch 19, Loss: 0.0384263015137269, Train Accuracy: 0.9580201443174985, Test Accuracy: 0.9485868911605532


100%|██████████| 20/20 [00:55<00:00,  2.77s/it]

Epoch 20, Loss: 0.037848459699979194, Train Accuracy: 0.9586966325917018, Test Accuracy: 0.9493385447985568
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9594    0.9799    0.9695      5470
           1     0.8967    0.8080    0.8500      1182

    accuracy                         0.9493      6652
   macro avg     0.9280    0.8939    0.9098      6652
weighted avg     0.9482    0.9493    0.9483      6652



In [31]:
criterion = nn.MSELoss()
feature_column_name = 'number-annotation' # label, Trend, Layout, number annotation, grouping

features = resnet_features
classifier = train(data, features, feature_column_name, criterion)

  5%|▌         | 1/20 [00:02<00:56,  3.00s/it]

Epoch 1, Loss: 0.5378741438572223, Train Accuracy: 0.7797279013830427, Test Accuracy: 0.8457606734816596


 10%|█         | 2/20 [00:05<00:50,  2.83s/it]

Epoch 2, Loss: 0.14398399797769693, Train Accuracy: 0.8497068550811786, Test Accuracy: 0.8693625977149729


 15%|█▌        | 3/20 [00:08<00:45,  2.69s/it]

...

Epoch 18, Loss: 0.07222064011372052, Train Accuracy: 0.9114176187612748, Test Accuracy: 0.9099518941671678


 95%|█████████▌| 19/20 [00:53<00:02,  2.84s/it]

Epoch 19, Loss: 0.07133303582668304, Train Accuracy: 0.9126202645820806, Test Accuracy: 0.908749248346362


100%|██████████| 20/20 [00:56<00:00,  2.82s/it]

Epoch 20, Loss: 0.07033189901938805, Train Accuracy: 0.914987973541792, Test Accuracy: 0.9101022248947684
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9239    0.9706    0.9467      5470
           1     0.8223    0.6303    0.7136      1182

    accuracy                         0.9101      6652
   macro avg     0.8731    0.8004    0.8301      6652
weighted avg     0.9059    0.9101    0.9053      6652



In [20]:
criterion = nn.MSELoss()
feature_column_name = 'number-annotation' # label, Trend, Layout, number annotation, grouping

features = clip_features
classifier = train(data, features, feature_column_name, criterion)

  5%|▌         | 1/20 [00:02<00:54,  2.85s/it]

Epoch 1, Loss: 0.1843610514815037, Train Accuracy: 0.8177239927841251, Test Accuracy: 0.8541791942273


 10%|█         | 2/20 [00:05<00:49,  2.78s/it]

Epoch 2, Loss: 0.11021389353733796, Train Accuracy: 0.86458959711365, Test Accuracy: 0.8672579675285629


 15%|█▌        | 3/20 [00:08<00:47,  2.82s/it]

...

Epoch 18, Loss: 0.04464701763712443, Train Accuracy: 0.9478352375225496, Test Accuracy: 0.9374624173180999


 95%|█████████▌| 19/20 [00:49<00:02,  2.67s/it]

Epoch 19, Loss: 0.043812879003011264, Train Accuracy: 0.9493385447985568, Test Accuracy: 0.9356584485868912


100%|██████████| 20/20 [00:52<00:00,  2.62s/it]

Epoch 20, Loss: 0.043180872614567094, Train Accuracy: 0.9497519542994588, Test Accuracy: 0.9388153938665064
Classification Report on Test Data:
              precision    recall  f1-score   support

           0     0.9544    0.9720    0.9631      5470
           1     0.8585    0.7851    0.8202      1182

    accuracy                         0.9388      6652
   macro avg     0.9064    0.8786    0.8916      6652
weighted avg     0.9374    0.9388    0.9377      6652

